AI Ranger Team Demo Development
# Deep Learning for Medical Image Analysis leveraging the AML Platform
### Pneunomia detection using Remote Experiment Runs and Azure ML Pipeines

<img src="images/medicalimage.jpg" width=1000 />

### In this notebook
In this notebook, three different approaches are demonstrated to train a Pneumonia detection model using the unique capabilities of Azure ML. We will start by training a baseline model, which is trained on a remote cluster with GPU machines. The second part will show an approach for training hyperparameters. The third and last part of the notebook, will transform the Remote Experiment Runs in a Azure ML Pipeline, that can be used to train a model repeatably when new data is available. The Pipeline will also include a step for deployment.

# Pneumonia Detection Use Case
A relatively small public dataset of medical images for detecting viral or bacterial pneumonia has been chosen to keep the scenario straightforward and reproducible with limited computing resources. The dataset contains 5,218 x-ray images with two classes of diagnostic outcomes: 3,876 cases with (viral or bacterial) pneumonia and 1,342 cases without findings ("Normal").
The dataset is split into training, validation and test sets. Since some images represent radiographs from the same patient, it has been ensured that there is no overlap of patients between the training, validation and test sets.

<img src="images/pneumonia.png" width=1000 />
You can find the dataset under this location: https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia. 


# Neural Network architecture

The following neural network architecture is used:

<img src="images/cnnframe.png" width=1200 />

Though a detailed discussion of the architecture and functionality of Convnets is outside the scope of this demo, the following summary provides a brief overview ofthe design:
- The x-ray images are resized to a 224 x 224 pixel resolution before being fed into the Convnet. Other medical imaging use cases will most likely require higher resolutions. However, for the selected dataset, high accuracy results can be achieved with this small image size.
- During  the  data  flow  through  the Convnet, relevant  properties  for  the  classification  task (features) are extracted in a hierarchical way. The lower layers of the network detect low-level features like edges or surfaces. More complex features (for detecting pneumonia in this case) are extracted at higher layers. The three convolutional layers perform the detection of features at different abstraction levels in the network, where the images are scanned by a small moving window (kernel).
- To reduce computational effort while focusing on the most dominant features, the image size is reduced further as the data flows through the three max pooling layers.
- Two dropout layers are included to reduce the risk of overfitting to the training data.
- The final layer consists of two neurons for representing the classes "pneumonia" and "normal".

# Setup


## Installs and imports

In [ ]:
# Download Kaggle pip package and split-folders
%pip install kaggle --upgrade split-folders

In [1]:
import matplotlib.pyplot as plt

import json
from azureml.core import Workspace, Dataset, Experiment

workspace = Workspace.from_config()

## Retrieve and upload data

When you use this notebook for the first time, the pneumonia dataset should be uploaded to the default AzureML datastore and registered as a managed file dataset.

The commands below can be used to download the dataset using the Kaggle API (https://github.com/Kaggle/kaggle-api). Use the instructions to generate your own API key and fill them in on the code cell.

In [ ]:
# Export Kaggle configuration variables
%env KAGGLE_USERNAME=[Kaggle user name]
%env KAGGLE_KEY=[API token]

In [2]:
# Export Kaggle configuration variables
%env KAGGLE_USERNAME=ankopp
%env KAGGLE_KEY=7400527eb41ba5d3248625bea6f1a3bb

env: KAGGLE_USERNAME=ankopp
env: KAGGLE_KEY=7400527eb41ba5d3248625bea6f1a3bb


In [3]:
# remove folders and zipfile from previous runs of the cell
!rm /tmp/chest-xray-pneumonia.zip
!rm -r /tmp/chest_xray
!rm -r /tmp/chest_xray_tvt

# Download the Pneumonia dataset
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia -p /tmp

!unzip -q /tmp/chest-xray-pneumonia.zip -d /tmp

rm: cannot remove '/tmp/chest_xray_tvt': No such file or directory
100%|██████████████████████████████████████▉| 2.29G/2.29G [00:12<00:00, 189MB/s]
100%|███████████████████████████████████████| 2.29G/2.29G [00:12<00:00, 205MB/s]


In [4]:
import splitfolders

download_root = '/tmp/chest_xray/train' 
train_val_test_root = '/tmp/chest_xray_tvt/'

train_val_test_split = (0.8, 0.1, 0.1)
random_seed = 33

splitfolders.ratio(download_root, train_val_test_root, random_seed, ratio=train_val_test_split)

Copying files: 5216 files [00:02, 1796.35 files/s]


In [5]:
# check dataset splits
for split in os.listdir(train_val_test_root):
    for label in ['NORMAL', 'PNEUMONIA']:
        files = os.listdir(os.path.join(train_val_test_root, split, label))
        print(f'{split}-{label}: ', len(files))


val-NORMAL:  134
val-PNEUMONIA:  387
train-NORMAL:  1072
train-PNEUMONIA:  3100
test-NORMAL:  135
test-PNEUMONIA:  388


In [6]:
from azureml.core import Workspace, Datastore, Dataset
from azureml.data.datapath import DataPath

# Upload data to AzureML Datastore
ds = workspace.get_default_datastore()
ds = Dataset.File.upload_directory(src_dir=train_val_test_root,
            target=DataPath(ds, 'chest-xray'),
            show_progress=False, overwrite=False)

# Register file dataset with AzureML
ds = ds.register(workspace=workspace, name="pneumonia", description="Pneumonia train / val / test folders with 2 classes", create_new_version=True)

print(f'Dataset {ds.name} registered.')

Dataset pneumonia registered.


# I. Run baseline experiment

## Create/retrieve Compute Cluster

In [7]:
from azureml.core.compute import AmlCompute, ComputeTarget

cluster_name = "gpu-cluster"

try:
    compute_target = workspace.compute_targets[cluster_name]
    print('Found existing compute target.')
except KeyError:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_NC6', 
                                                           idle_seconds_before_scaledown=1800,
                                                           min_nodes=0, 
                                                           max_nodes=4)

    compute_target = ComputeTarget.create(workspace, cluster_name, compute_config)
    
# Can poll for a minimum number of nodes and for a specific timeout.
# If no min_node_count is provided, it will use the scale settings for the cluster.
compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

Found existing compute target.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


### Define ScriptRunConfig object

Define the Environment that you will use to run your experiment, retrieve the dataset by name and define the ScriptRunConfig object.

In [8]:
from azureml.core import ScriptRunConfig, Environment
from azureml.core.compute import ComputeTarget

experiment = Experiment(workspace, 'pneumonia')

pytorch_env = Environment.from_conda_specification(name = 'pytorch-1.6-gpu', file_path = './training/conda_dependencies.yml')

dataset = Dataset.get_by_name(workspace, name='pneumonia', version='latest')

src = ScriptRunConfig(source_directory='./training',
                      script='train.py',
                      arguments=['--epochs', 15, '--data-folder', dataset.as_mount()],
                      compute_target= ComputeTarget(workspace, 'gpu-cluster'),
                      environment=pytorch_env)

## Submit baseline experiment

In [9]:
from azureml.widgets import RunDetails

script_run = experiment.submit(src)
RunDetails(script_run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

# II. Hyperparameter tuning using Random Parameter Sampling
Hyperparameter tuning, also called hyperparameter optimization, is the process of finding the configuration of hyperparameters that results in the best performance. The process is typically computationally expensive and manual.

Azure Machine Learning lets you automate hyperparameter tuning and run experiments in parallel to efficiently optimize hyperparameters.

Random sampling supports discrete and continuous hyperparameters. It supports early termination of low-performance runs. Some users do an initial search with random sampling and then refine the search space to improve results. In random sampling, hyperparameter values are randomly selected from the defined search space.

Selected hyperparameters affect various stages of the experiment:

- Data: Training and validation loader: batch size
- CNN Architecture: Dropout
- Choice of optimizer
- Training loop: learning rate

In [20]:
from azureml.train.hyperdrive import RandomParameterSampling, BanditPolicy, HyperDriveConfig, uniform, choice, PrimaryMetricGoal

param_sampling = RandomParameterSampling( {
        'learning_rate': choice(0.00007, 0.0007, 0.07),
        'batch_size': choice(16, 32, 64, 128), 
        'conv_dropout' : uniform(0.0, 0.5), 
        'optimizer': choice('SGD', 'Adam', 'RMSprop')
    }
)

early_termination_policy = BanditPolicy(slack_factor=0.15, evaluation_interval=1, delay_evaluation=5)

hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=param_sampling, 
                                     policy=early_termination_policy,
                                     primary_metric_name='best_val_acc',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=8,
                                     max_concurrent_runs=4)

NameError: name 'src' is not defined

## Submit hyperdrive run

In [23]:
from azureml.widgets import RunDetails

# start the HyperDrive run
hyperdrive_run = experiment.submit(hyperdrive_config)

RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

#  III. Define AML pipeline with HyperDriveStep

The third part of this, is a showcase on how to use the Azure ML Pipeline capability to create a pipeline from the same training script that we have been using. In the cell below, the first step of the pipeline is created, by defining the pipeline data that will be the output of the first step.
The Hyperdrive config that we have defined in the previous step will be re-used.

In [19]:
from azureml.pipeline.steps import HyperDriveStep, HyperDriveStepRun, PythonScriptStep
from azureml.pipeline.core import Pipeline, PipelineData, TrainingOutput

metrics_output_name = 'metrics_output'
metrics_data = PipelineData(name='metrics_data',
                            datastore=workspace.get_default_datastore(),
                            pipeline_output_name=metrics_output_name,
                            training_output=TrainingOutput("Metrics"))

model_output_name = 'model_output'
saved_model = PipelineData(name='saved_model',
                            datastore=workspace.get_default_datastore(),
                            pipeline_output_name=model_output_name,
                            training_output=TrainingOutput("Model",
                                                           model_file="outputs/model/pneumonia.pt"))

hd_step_name='hyperdrive_step'
hd_step = HyperDriveStep(
    name=hd_step_name,
    hyperdrive_config=hyperdrive_config,
    inputs=[dataset.as_mount()],
    outputs=[metrics_data, saved_model])


NameError: name 'hyperdrive_config' is not defined

## Find and register best model

We add a step in our pipeline to find and register the best model, that is the output of the Hyperdrivestep.

In [27]:
%%writefile training/register_model.py

import argparse
import json
import os
from azureml.core import Workspace, Experiment, Model
from azureml.core import Run
from shutil import copy2

parser = argparse.ArgumentParser()
parser.add_argument('--saved-model', type=str, dest='saved_model', help='path to saved model file')
args = parser.parse_args()

model_output_dir = './model/'

os.makedirs(model_output_dir, exist_ok=True)
copy2(args.saved_model, model_output_dir)

ws = Run.get_context().experiment.workspace

model = Model.register(workspace=ws, model_name='tf-dnn-mnist', model_path=model_output_dir)

Writing training/register_model.py


In [28]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies

conda_dep = CondaDependencies()
conda_dep.add_pip_package("azureml-sdk")

rcfg = RunConfiguration(conda_dependencies=conda_dep)

register_model_step = PythonScriptStep(source_directory='./training',
                                       script_name='register_model.py',
                                       name="register_model_step01",
                                       inputs=[saved_model],
                                       compute_target=ComputeTarget(workspace, 'gpu-cluster'),
                                       arguments=["--saved-model", saved_model],
                                       allow_reuse=True,
                                       runconfig=rcfg)

register_model_step.run_after(hd_step)

## Submit pipeline including model registration

In [29]:
pipeline = Pipeline(workspace=workspace, steps=[hd_step, register_model_step])
pipeline_run = experiment.submit(pipeline)

Created step hyperdrive_step [bb3d6f60][c4622312-5a75-4323-b72f-7652219f234a], (This step will run and generate new outputs)
Created step register_model_step01 [72891fc7][e63ede12-f73e-4f31-a1d1-7382d70ecf4f], (This step will run and generate new outputs)
Submitted PipelineRun 849bb7c8-d0b4-4072-bba7-33d21323b499
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/849bb7c8-d0b4-4072-bba7-33d21323b499?wsid=/subscriptions/4eeedd72-d937-4243-86d1-c3982a84d924/resourcegroups/livecell/workspaces/livecell&tid=72f988bf-86f1-41af-91ab-2d7cd011db47


## Download training metrics

In [31]:
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)


Downloaded azureml/1ac75806-bf42-4bb9-ae37-511878acc23a/metrics_data, 1 files out of an estimated total of 1


## Visualize training metrics

In [32]:
import pandas as pd
import json
with open(metrics_output._path_on_datastore) as f:  
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

,HD_94cc904c-c26f-47bc-93de-27303845914e_0,HD_94cc904c-c26f-47bc-93de-27303845914e_3,HD_94cc904c-c26f-47bc-93de-27303845914e_2,HD_94cc904c-c26f-47bc-93de-27303845914e_1,HD_94cc904c-c26f-47bc-93de-27303845914e_6,HD_94cc904c-c26f-47bc-93de-27303845914e_5,HD_94cc904c-c26f-47bc-93de-27303845914e_4,HD_94cc904c-c26f-47bc-93de-27303845914e_7
best_val_acc,"[0.73828125, 0.94140625, 0.94140625, 0.9550781...","[0.947265625, 0.94921875, 0.94921875, 0.970703...","[0.291015625, 0.73828125, 0.73828125, 0.738281...","[0.935546875, 0.9453125, 0.955078125, 0.972656...","[0.923828125, 0.9375, 0.9453125, 0.955078125, ...","[0.73828125, 0.90234375, 0.923828125, 0.925781...","[0.931640625, 0.955078125, 0.978515625, 0.9785...","[0.96875, 0.97265625, 0.97265625, 0.97265625, ..."
validation accuracy,"[0.73828125, 0.94140625, 0.9296875, 0.95507812...","[0.947265625, 0.94921875, 0.943359375, 0.97070...","[0.291015625, 0.73828125, 0.73828125, 0.738281...","[0.935546875, 0.9453125, 0.955078125, 0.972656...","[0.923828125, 0.9375, 0.9453125, 0.955078125, ...","[0.73828125, 0.90234375, 0.923828125, 0.925781...","[0.931640625, 0.955078125, 0.978515625, 0.9746...","[0.96875, 0.97265625, 0.970703125, 0.97265625,..."
training accuracy,"[0.7314903846153846, 0.7838942307692308, 0.872...","[0.865625, 0.9552884615384616, 0.9620192307692...","[0.71630859375, 0.71826171875, 0.744873046875,...","[0.8492788461538462, 0.9444711538461539, 0.958...","[0.80078125, 0.932861328125, 0.952880859375, 0...","[0.7418269230769231, 0.7759615384615385, 0.911...","[0.7822265625, 0.957763671875, 0.96875, 0.9760...","[0.9278846153846154, 0.9711538461538461, 0.976..."
validation loss,"[0.5553653775616022, 0.2465433172895904, 0.245...","[0.14292823055655393, 0.1570459528756462, 0.17...","[0.6951569247840691, 0.5608080023767394, 0.529...","[0.16253205132804768, 0.13889057942864533, 0.1...","[0.351493095939768, 0.18049384170210064, 0.173...","[0.5273191192099778, 0.3954645850608079, 0.261...","[0.14793442123910974, 0.10387369492690074, 0.0...","[0.06922712095525123, 0.06690031610386385, 0.0..."
Train imgs,[4172.0],[4172.0],[4172.0],[4172.0],[4172.0],[4172.0],[4172.0],[4172.0]
training loss,"[0.562905370560497, 0.39944255797769285, 0.340...","[0.37048336294902023, 0.12536239506871452, 0.1...","[0.5913689239370286, 0.5866566563841252, 0.540...","[5.801720656861738, 0.16121490541149078, 0.110...","[0.6851767497003249, 0.18859169192876477, 0.12...","[0.5608543389595595, 0.4458411873129847, 0.275...","[0.5561550763057954, 0.11649330224652532, 0.08...","[0.19159154064974757, 0.07936467543380087, 0.0..."


## Publish the training pipeline

By publishing the training pipeline, an pipeline endpoint is created, that we can use to trigger the pipeline from external services.

In [33]:
published_pipeline1 = pipeline_run.publish_pipeline(
     name="Training_pneumonia",
     description="Pipeline to train a classification model to detect pneumonia.",
     version="1.0")

In [36]:
published_pipeline1.id

'4682df08-14d3-4a8b-9d61-8434ae3f8842'

## Create a schedule based on file change
One advantage of defining and publishing your training script as an Azure ML Pipeline, is that a schedule can be created to trigger retraining of your model based on file changes in the source dataset.

In [14]:
from azureml.pipeline.core.schedule import Schedule

schedule = Schedule.list(workspace) 
print(schedule)

sch = Schedule.list(workspace)[0] # I want to disable the first pipeline
    
Schedule.disable(sch)

[Pipeline(Name: MyReactiveSchedule,
Id: 96c9feb6-8ea5-42f3-9056-562337fcf226,
Status: Active,
Pipeline Id: 4682df08-14d3-4a8b-9d61-8434ae3f8842,
Pipeline Endpoint Id: None,
Datastore: workspaceblobstore)]


In [18]:
from azureml.pipeline.core import PipelineEndpoint

PipelineEndpoint.list(workspace)

# PipelineEndpoint.get(workspace=workspace, name="Training_pneumonia")

[]

In [37]:
from azureml.pipeline.core.schedule import Schedule
from azureml.pipeline.core import PipelineEndpoint

datastore = workspace.get_default_datastore()

pipeline_endpoint_by_name = PipelineEndpoint.get(workspace=workspace, name="Training_pneumonia")

reactive_schedule = Schedule.create(workspace, name="MyReactiveSchedule", 
                                    description="Based on input file change.",
                                    pipeline_id=pipeline_endpoint_by_name.id,
                                    experiment_name='experiment_name',
                                    datastore=datastore, 
                                    path_on_datastore="chest-xray/train/PNEUMONIA"
                                    data_path_parameter_name="input_data")


ErrorResponseException: (BadRequest) Response status code does not indicate success: 400 (Bad Request).
Microsoft.RelInfra.Common.Exceptions.ErrorResponseException: PipelineEndpoint name Training_pneumonia not found in workspace a87d11b1-1984-4762-89e7-11e38d93e528

In [17]:
import os
import shutil

retrain_source = '/tmp/chest_xray/test'
target_samples = './chest-xray-retrain'

for label in ['NORMAL', 'PNEUMONIA']:
    files = os.listdir(os.path.join(retrain_source, label))[:5]
    for filename in files:
        source = os.path.join(retrain_source, label, filename)
        target = os.path.join(target_samples, label, 'add_'+ filename)
        shutil.copyfile(source, target)


In [14]:
name = 'NORMAL2-IM-0146-0001.jpeg'

label = 'NORMAL'
source = os.path.join('/tmp/chest_xray/test',label, name)
target = os.path.join('./chest-xray-retrain',label,'add_'+name)

print(source)
print(target)

shutil.copyfile(source, target)

/tmp/chest_xray/test/NORMAL/NORMAL2-IM-0146-0001.jpeg
./chest-xray-retrain/NORMAL/add_NORMAL2-IM-0146-0001.jpeg


'./chest-xray-retrain/NORMAL/add_NORMAL2-IM-0146-0001.jpeg'

In [20]:
from azureml.data.datapath import DataPath

ds = workspace.get_default_datastore()
ds = Dataset.File.upload_directory(src_dir=target_samples,
            target=DataPath(ds, 'chest-xray/train'),
            show_progress=True, overwrite=False)


Validating arguments.
Arguments validated.
Uploading file to chest-xray/train
Uploading an estimated of 10 files
Target already exists. Skipping upload for chest-xray/train/NORMAL/add_IM-0077-0001.jpeg
Target already exists. Skipping upload for chest-xray/train/NORMAL/add_NORMAL2-IM-0146-0001.jpeg
Target already exists. Skipping upload for chest-xray/train/NORMAL/add_NORMAL2-IM-0198-0001.jpeg
Target already exists. Skipping upload for chest-xray/train/NORMAL/add_NORMAL2-IM-0241-0001.jpeg
Target already exists. Skipping upload for chest-xray/train/NORMAL/add_NORMAL2-IM-0276-0001.jpeg
Target already exists. Skipping upload for chest-xray/train/PNEUMONIA/add_person133_bacteria_637.jpeg
Target already exists. Skipping upload for chest-xray/train/PNEUMONIA/add_person1628_virus_2821.jpeg
Target already exists. Skipping upload for chest-xray/train/PNEUMONIA/add_person1676_virus_2892.jpeg
Target already exists. Skipping upload for chest-xray/train/PNEUMONIA/add_person1_virus_13.jpeg
Target alr

In [4]:
from azureml.core import Dataset
import tempfile

dataset = Dataset.get_by_name(workspace, name='pneumonia')

mounted_path = tempfile.mkdtemp()

# mount dataset onto the mounted_path of a Linux-based compute
mount_context = dataset.mount(mounted_path)

mount_context.start()

import os
print(os.listdir(mounted_path))
print (mounted_path)

['test', 'train', 'val']
/tmp/tmpx9wwwjkz


In [9]:
train_root = os.path.join(mounted_path, 'train')

for label in ['NORMAL', 'PNEUMONIA']:
    add_files = [file for file in os.listdir(os.path.join(train_root, label)) if file.startswith('add_')]
    print(add_files)
    # delete added file
    for file in add_files:
        os.remove(os.path.join(train_root, label, file))




['add_IM-0077-0001.jpeg', 'add_NORMAL2-IM-0146-0001.jpeg', 'add_NORMAL2-IM-0198-0001.jpeg', 'add_NORMAL2-IM-0241-0001.jpeg', 'add_NORMAL2-IM-0276-0001.jpeg']


OSError: [Errno 30] Read-only file system: '/tmp/tmpx9wwwjkz/train/NORMAL/add_IM-0077-0001.jpeg'